<a href="https://colab.research.google.com/github/JosephAlves0/dio-machine-learning-training/blob/main/DarknetToColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Notebook Title](http://blog.ibanyez.info/download/B20190408T000000071.jpg)

# How to train YOLOv3 using Darknet on Colab notebook and optimize the VM runtime load times

## Welcome!

This Colab notebook will show you how to:

* Train a **Yolo v3** model using **Darknet** using the Colab **12GB-RAM GPU**.
* Turn Colab notebooks into an effective tool to work on real projects. Dealing with the handicap of a runtime that will **blow up every 12 hours** into the space!
  * Working directly from the files on your computer.
  * Configure your notebook to install everything you need and start training in about a minute (Tested using 550MB dataset).
  * Receive your trained weights directly on your computer during the training. While the notebook is training you can check how it is going using your trained weights in your computer.


#### This notebook is part of the post [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](http://blog.ibanyez.info/blogs/coding/20190410-run-a-google-colab-notebook-to-train-yolov3-using-darknet-in/) I encourage you to visit! You will find a deeper explanation about Google Colab, the goods and the limitations of this great tool.


### These are the steps we'll follow:

* Configure Google Drive and map as network  _Drive_.
* Some utils to help to do some tasks.
* Configure the pre-requisites on the runtime.
  * Check the CUDA installation on the runtime VM.
  * Install cuDNN.
  * Clone and compile Darknet. We'll use a repo based on [AlexeyAB's Darknet repo](https://github.com/AlexeyAB/darknet/). I applied  some changes to make possible to load files from `/My Drive/` and reduced the number of logs on console to speed up the notebook.
  * We'll check that everything works great.
* Explained how to manage your YOLO files on your computer and it will be used transparently from this notebook.

> _**NOTE:** Cells with an annotation **`# Not Necessary cell`**. Can be removed without having any impact. They are only explanatory content._

### Without further ado, let's start!


## STEP 0. Configure runtime to work with GPU

We want to use the **12GB-RAM GPU** hardware acceleration!

Go to **> Menu > Runtime > Configure Runtime Type** And select **GPU** From the **Hardware accelerator** drop down meu

## STEP 1. Connect your files to Google Drive
In order to have your files in your local computer you need to install and configure Google Backup and Sync to keep one folder of your Drive synced with a folder on your computer.

![schema drive.jpg](http://blog.ibanyez.info/download/B20190408T000000060.jpg)

1. Create a folder on your Google Drive named _**darknet**_
2. Configure the Google Backup and Sync as follows. If you don't speak Spanish, you maybe learn some words! **;)**
![Sync Drive.jpg](http://blog.ibanyez.info/download/B20190408T000000063.jpg)

**After this step you'll have a folder called _darknet_ in your local computer. This folder is where you will work with files on your computer**

> _**TIP** - We need to have a good performance downloading data from Drive to Colab. Having a lot of files in your Drive root folder can slow down the things quite a bit. It's a good practice working with Colab to move all your root folder files into a folder_




## STEP 2. Connect the Colab notebook to Google Drive

Now we're gonna map your Google Drive folder. This first step is the only one that will require your manual interaction every time you run your notebook.

* Execute the following cell _(Click on Play button or press CTRL + ENTER)_ and click on the link to authorize your notebook to access to your Google Drive.
* Paste the code Google will give to you and push `enter`

In [ ]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. You can access to your Drive files
# using this path "/content/gdrive/My Drive/"

from google.colab import drive
drive.mount('/content/gdrive')

Congratulations! Now you can access to your local computer folder directly from here!

Check it out!

In [ ]:
# Not Necessary cell
# List the content of your local computer folder
!ls -la "/content/gdrive/My Drive/darknet"

In [ ]:
!sudo apt-get install tree

In [ ]:
!tree /content/gdrive/My\ Drive/darknet/

### UPDATE NOTE

If you want to simplify your paths, you can use a Symbolic link:

`!ln -s "/content/gdrive/My Drive/darknet/" /mydrive`

 Then you'll be able to access your Google Drive files just using `/mydrive` path

**BE CAREFUL: All the paths in this notebook are without using Symbolic link. You'll have to remember to change the path everywhere**

Thanks to **Dennis Kashkin** for this recommendation!

In [ ]:
# Uncomment if you want to use Symbolic link
#!ln -s /content/gdrive/My\ Drive/darknet/ /mydrive
#!ls /mydrive

## STEP 2. Check CUDA release version

Nvidia CUDA is pre-installed on Colab notebooks. Now we'll check the version installed.

> _**BE AWARE:** Maybe some time from the time I'm writing these lines (April 9th, 2019)  the CUDA version is upgraded on Colab and you should download another version of the cuDNN in the next step. Now is release 10.0 and we are using cuDNN (cudnn-10.0-linux-x64-v7.5.0.56.tgz) accordingly_

In [ ]:
# This cell can be commented once you checked the current CUDA version
# CUDA: Let's check that Nvidia CUDA is already pre-installed and which version is it. In some time from now maybe you
!/usr/local/cuda/bin/nvcc --version


## STEP 3. Install cuDNN according to the current CUDA version
Now, you need to download cuDNN from Nvidia web site. You'll need to sign up on the site.

* Download cuDNN from [Nvidia website](https://developer.nvidia.com/cudnn)

  * Right now, because we have _**CUDA 10.0**_ preinstalled in Colab runtime, you need download [cuDNN v7.5.0.56 for CUDA v10.0](https://developer.nvidia.com/compute/machine-learning/cudnn/secure/v7.5.0.56/prod/10.0_20190219/cudnn-10.0-linux-x64-v7.5.0.56.tgz) - the file is cudnn-10.0-linux-x64-v7.5.0.56.tgz

* On your local computer, create a folder named _**cuDNN**_ in your local folder _**darknet**_. Copy the _**tgz**_ file there



In [ ]:
# We're unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
!tar -xzvf gdrive/My\ Drive/darknet/cuDNN/cudnn-10.0-linux-x64-v7.5.0.56.tgz -C /usr/local/
!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

## STEP 4. Installing Darknet
Great!! We have all the necessary to start working with Darknet.

This notebook works with a slightly modified version of darknet, which is based on the [AlexeyAB Darknet repo](https://github.com/AlexeyAB/darknet/)
The changes applied to the original repo are:
* Allowing to use spaces on the darknet configuration files _**obj.data**_. Necessary to work with Google Drive directly.
* Removing some logs on every epoch. The original repo write more logs than the Colab notebook can sync. This creates a long queue during the training. This version only shows the results after every iteration.

You can take a look to the code at the [github repo](https://github.com/kriyeng/darknet/)

> _**TRICK**: Because we want to run the notebook fast every time we will compile darknet only the first time we run this notebook. Then, we'll save the compiled version to your drive. For the future executions we'll copy the compiled one instead of compiling again._


**Here comes our first trick to speed up the runtime load time**

* The first time we will:
  * Clone and compile the darknet project.
  * Copy the compiled version to our Google Drive Floder
  
* The next times, instead of compiling it again on every runtime load, we'll copy the compiled version to our VM machine!

When compiling the first time, your output last line has to be something like this:

`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)`

## STEP 4-A. Cloning and compiling Darkent. ONLY NEEDS TO BE RUN ON THE FIRST EXECUTION!!
In this step we'll clone the darkent repo and compile it.
* Clone Repo
* Compile Darknet
* Copy compiled version to Drive

When compiling ends, your output last line has to be something like this:

_`g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv` -DGPU (...)_`

In [ ]:
# Leave this code uncommented on the very first run of your notebook or if you ever need to recompile darknet again.
# Comment this code on the future runs.
!git clone https://github.com/kriyeng/darknet/
%cd darknet

# Check the folder
!ls

# I have a branch where I have done the changes commented above
!git checkout feature/google-colab

#Compile Darknet
!make

#Copies the Darknet compiled version to Google drive
!cp ./darknet /content/gdrive/My\ Drive/darknet/bin/darknet

## STEP 4-B. Copying the compiled version of Darknet from Drive. UNCOMMENT AFTER FIRST EXECUTION
Copy the darknet compiled version from drive to the VM.
* Make the local darknet folder
* Copy the darknet file
* Set execution permissions

In [ ]:
# Uncomment after the first run, when you have a copy of compiled darkent in your Google Drive

# Makes a dir for darknet and move there
#!mkdir darknet
#%cd darknet

# Copy the Darkent compiled version to the VM local drive
#!cp /content/gdrive/My\ Drive/darknet/bin/darknet ./darknet

# Set execution permissions to Darknet
#!chmod +x ./darknet


## STEP 5. Runtime configuration finished!
Let's chek it out!

If you are running this notebook for the first time, you can run the following cells in order to check if everything goes as expected!




## Some Utils
Let's add some utils that maybe can be useful.

These utils are:
* imgShow() - Will help us to show an image in the remote VM
* download() - Will allow you to get some file from your notebook in case you need to
* upload() - You can upload files to your current folder on the remote VM.

Thanks to [Ivan Goncharov](https://twitter.com/Ivangrov) for these helpers!

In [ ]:
#download files
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  #plt.rcParams['figure.figsize'] = [10, 5]
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()


def upload():
  from google.colab import files
  uploaded = files.upload()
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# Not necessary cell
# Get yolov3 weights
!wget https://pjreddie.com/media/files/yolov3.weights

**NOTE:** The following test only will work when the darknet is compiled in the runtime. This demo uses some data from the original darknet folders. For your Object detection projects, you'll have these necessary files on your local folder.

In [ ]:
# Not necessary cell
# Execute darknet using YOLOv3 model with pre-trained weights to detect objects on 'person.jpg'
!./darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show

# Show the result using the helper imgShow()
imShow('predictions.jpg')

## If you can see the same picture as the one below, congratulations!! At this point you have Darknet configured and working!

![person.jpg](http://blog.ibanyez.info/download/B20190409T000000064.png)

# PART 2. Training YOLO

 > _**TRICK:** Every time you want to run all your cells automatically you can go to the **> Menu > Runtime > run all**. Maybe you don't want to execute the entire notebook. You can write the following cell where you want to stop the process and uncoment the **`assert False`** command. This will throw an error and will stop to run more cells. Thanks to: [This thread](https://groups.google.com/forum/#!topic/jupyter/ELftSFSiedQ)_




In [ ]:
# don't go beyond here with Run All
#assert False

## PART 2. STEP 0. Preparing your data and configuration files

Before going further let's take a look at what configuration files you need to have in your local drive _`darknet`_

![Yolov3 configuration files cheat sheet.jpg](http://blog.ibanyez.info/download/B20190410T000000072.png)

You can download the cheat sheet [here](http://blog.ibanyez.info/download/B20190410T000000072.png)

If you need deeper explanations on how to prepare your data sets, annotation and deep learning, visit [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)


> **TRICK:** You have to be carefully configuring paths on your config files. _*obj.data*_ file needs to have spaces on the path escaped with _**\**_. Like this: **_/content/gdrive/My\ Drive/darknet/train.txt_**. But, in files **_train.txt_** and **_test.txt_** does not!




## PART 2. STEP 1. Loading files to VM local drive
The network speed between Google Drive and Colab VM can have an impact on your training speed accessing to your dataset images.

You can have 3 possible approaches depending on the size of your dataset.

> _**NOTE:** This step is not necessary for all the configuration files and weights. They can be accessed directly from Google Drive without considerable performance issues. **This step is only for the dataset images and annotations**_

* **Option 1** - You can try  to use directly the files from Google Drive _`img/`_ folder. Depending on your dataset maybe this can be more than good.
* **Option 2** - Before start training copy your dataset from Google Drive to the local VM filesystem. Maybe can be a good practice to copy as one single tar file and decompress in your VM local _`img/`_ folder
* **Option 3** - If your dataset is quite big, maybe you can upload to a git repository and clone from here. Usually transfer time between are much better. If you have to decide I have the feeling that bitbucket have better speed transfer times than github, but please, don't take this as confirmed, **I haven't done specific tests on that, I could be wrong!**




#### PART 2. STEP 1 - Option 1. Using files from Google Drive directly.
You don't have to do anything here. Your **_train.txt_** should have the correct path:
* **/content/grdive/My Drive/darknet/img/image001.jpg**. As said before, don't use escaped white space for the paths on _**train.txt**_ and _**test.txt**_

#### PART 2. STEP 1 - Option 2A. Copying files from Google Drive to VM local filesystem.
Execute the follow cell to copy your files

In [ ]:
# Copy fils from Google Drive to the VM local filesystem
!cp -r "/content/gdrive/My Drive/darknet/img" ./img


#### PART 2. STEP 1 - Option 2B. Copying files zipped from Google Drive to VM local filesystem and unzip locally.
Execute the follow cell to copy your files and uncompress.
You can use _*!ls*_ command to esnure what's the correct path you have to configure in your _*train.txt*_ to correctly access to your dataset images


In [ ]:
# Copy your compressed file
#!cp -r "/content/gdrive/My Drive/darknet/img/img.tgz" ./img

# Uncompress zipped file
#!tar -xzvf ./img/img.tgz

#### PART 2. STEP 1 - Option 3. Clone your image dataset from a git repo. Seems the fastest one.
Execute the follow cell to clone your dataset repo to VM local filesystem


In [ ]:
# Git clone directly to ./img folder
#!git clone https://[your-repository] ./img

# Check the result - Uncomment when you checked for speed up further runs
#!ls -la ./img

## PART 3. Finally, Train your model!

When you execute the following command, your model will start training.

You will have a log line per epoch. On each iteration you will see how your training is going.

> **TRICK: Darknet copies a backup of your trained weights every 100 iterations. As magic, this file will be synced to your local drive on your computer, as well as the backups darknet do every 1000 iterations, saving it on a separate file.**





In [ ]:
!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/darknet53.conv.74" -dont_show

## PERFORMANCE TIPS & TRICKS

* **Speed up load times of the runtime:** When everything is checked that works, you can remove cells or comment unnecessary lines of code to make your loading time lower on every run.

* **How to keep your notebook alive for more time?:** Keep you browser with your notebook open. If you close your browser, your notebook will reach the iddle time and will be removed from Colab cloud Service. (90 minutes)
  
* **Re-run your training after reaching the limitation time for Colab runtimes (12 hours):**
  * Open a new notebook or reconnect the current one.
  * Comment the cell above and uncomment the cell below.
  * In your local computer, copy the file **backup/yolov3_last.weights** to your local computer **weights/** folder.
  * Execute Run all in the **> menu > Runtime > Run All**
  * _The copy step is not absolutely necessary, but I like to keep a copy of the last training session and not overwrite this file on next trainings._

In [ ]:
# Start training at the point where the last runtime finished
#!./darknet detector train "/content/gdrive/My Drive/darknet/obj.data" "/content/gdrive/My Drive/darknet/yolov3.cfg" "/content/gdrive/My Drive/darknet/weights/yolov_last.weights" -dont_show

## TROUBLESHOOTING
The main problems you can face if your model throw an error is:

* Images or files not found. Check the **Yolov3 cheat sheet** image above and check that everything is ok.
* If have you configured wrongly your **filters** and **classes** in the **yolov3.cfg**. Check the **Yolov3 cheet sheet** above.
* You can face some out of memory or library errors mainly for the lack of some the pre-requisits. In this case, check the versions of the current libraries installed on your Colab VM. You can find more information in the first steps of this notebook.
* **Batch** and **subdivisions** parameters on your **yolov3.cfg** can affect to the memory allocation as well. Refer to the original repo [Here]() for further details.

### TROUBLESHOOTING UPDATE
Be careful if you are preparing your files on Windows. If you use **CRLF** on your files instead of **LF** You can have problems opening the files correctly. - Thanks to [Satya Shetty](https://twitter.com/satyashetty) for sharing this issue!


## About me

You can find the original post with more explanations about this notebook at [How to train YOLOv3 using Darknet framework and optimize the VM runtime load times](post link)

I'm David Ibañez from Barcelona. Feel free to get in touch!

* You can visit my blog at [Dev-ibanyez.info](http://blog.ibanyex.info)
* You can get in touch with me on [Twitter](https://twitter.com/dav_ibanez)
* You can get in touch or contribute to this notebook at [Github](https://github.com/kriyeng/yolo-on-colab-notebook/)
* You can comment on the [dev.to post about this notebook ](PENDING)

Thanks for you having read this notebook! :clap: :clap: :clap:

## SOURCES


#### Other sources
* YOLO original web site [Joseph Redmon Page](https://pjreddie.com/darknet/yolo/)
* AlexeyAB darknet repo [github](https://github.com/AlexeyAB/darknet/)
* The Ivan Goncharov [notebook](https://github.com/ivangrov/YOLOv3-GoogleColab/blob/master/YOLOv3_GoogleColab.ipynb) inspired me to try Google Colab and end up creating this notebook.

In [ ]:
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar
!wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar

!tar xvf VOCtrainval_06-Nov-2007.tar
!tar xvf VOCtest_06-Nov-2007.tar
!tar xvf VOCtrainval_11-May-2012.tar

!mv VOCdevkit /content/VOCdevkit_2007
!mv VOCdevkit /content/VOCdevkit_2012

In [ ]:

!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar -O VOCtrainval_2007.tar
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar -O VOCtest_2007.tar

# Download Pascal VOC 2012 train/val
!wget http://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar -O VOCtrainval_2012.tar

# Create directories for extraction
!mkdir /content/VOCdevkit_2007
!mkdir /content/VOCdevkit_2012

# Extract the datasets to their respective directories
!tar xvf VOCtrainval_2007.tar -C /content/VOCdevkit_2007
!tar xvf VOCtest_2007.tar -C /content/VOCdevkit_2007
!tar xvf VOCtrainval_2012.tar -C /content/VOCdevkit_2012

# Verify the contents (optional)
!ls /content/VOCdevkit_2007/VOC2007/
!ls /content/VOCdevkit_2012/VOC2012/

In [ ]:
import xml.etree.ElementTree as ET
import os
import glob

classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(image_set, year, output_dir):
    in_path = f'/content/VOCdevkit_{year}/VOC{year}/Annotations/'
    out_path = output_dir
    image_list_path = f'/content/VOCdevkit_{year}/VOC{year}/ImageSets/Main/{image_set}.txt'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    with open(image_list_path, 'r') as f:
        image_ids = f.read().strip().split()

    for image_id in image_ids:
        in_file = open(in_path + image_id + '.xml')
        out_file = open(os.path.join(out_path, image_id + '.txt'), 'w')
        tree = ET.parse(in_file)
        root = tree.getroot()
        size = root.find('size')
        w = int(size.find('width').text)
        h = int(size.find('height').text)

        for obj in root.iter('object'):
            difficult = obj.find('difficult').text
            cls = obj.find('name').text
            if cls not in classes or int(difficult) == 1:
                continue
            cls_id = classes.index(cls)
            xmlbox = obj.find('bndbox')
            b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
            bb = convert((w,h), b)
            out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

        in_file.close()
        out_file.close()

output_annotations_dir = '/content/pascal_voc_yolo_annotations/'
if not os.path.exists(output_annotations_dir):
    os.makedirs(output_annotations_dir)


print("Converting VOC2007 trainval...")
convert_annotation('trainval', '2007', output_annotations_dir)
print("Converting VOC2007 test...")
convert_annotation('test', '2007', output_annotations_dir)
print("Converting VOC2012 trainval...")
convert_annotation('trainval', '2012', output_annotations_dir)

print("\nConversion complete!")
print(f"YOLO format annotations saved to: {output_annotations_dir}")

In [ ]:
import os

classes = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

yolo_annotations_dir = '/content/pascal_voc_yolo_annotations/'

darknet_dir = '/content/darknet/'

google_drive_darknet_dir = '/content/gdrive/My Drive/darknet/'

if not os.path.exists(os.path.join(google_drive_darknet_dir, 'bin')):
    os.makedirs(os.path.join(google_drive_darknet_dir, 'bin'))
if not os.path.exists(os.path.join(google_drive_darknet_dir, 'backup')):
    os.makedirs(os.path.join(google_drive_darknet_dir, 'backup'))

obj_names_path = os.path.join(darknet_dir, 'data', 'obj.names')
with open(obj_names_path, 'w') as f:
    for cls in classes:
        f.write(cls + '\n')

print(f"Arquivo {obj_names_path} criado.")

train_image_list = []
test_image_list = []

voc2007_trainval_path = '/content/VOCdevkit_2007/VOC2007/ImageSets/Main/trainval.txt'
with open(voc2007_trainval_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
        train_image_list.append(f'./img/VOC2007/JPEGImages/{img_id}.jpg')


voc2012_trainval_path = '/content/VOCdevkit_2012/VOC2012/ImageSets/Main/trainval.txt'
with open(voc2012_trainval_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
         train_image_list.append(f'./img/VOC2012/JPEGImages/{img_id}.jpg')


voc2007_test_path = '/content/VOCdevkit_2007/VOC2007/ImageSets/Main/test.txt'
with open(voc2007_test_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
        test_image_list.append(f'./img/VOC2007/JPEGImages/{img_id}.jpg')


train_txt_path = os.path.join(darknet_dir, 'data', 'train.txt')
with open(train_txt_path, 'w') as f:
    for img_path in train_image_list:
        f.write(img_path + '\n')

print(f"Arquivo {train_txt_path} criado com {len(train_image_list)} imagens.")

test_txt_path = os.path.join(darknet_dir, 'data', 'test.txt')
with open(test_txt_path, 'w') as f:
    for img_path in test_image_list:
        f.write(img_path + '\n')

print(f"Arquivo {test_txt_path} criado com {len(test_image_list)} imagens.")


obj_data_path = os.path.join(google_drive_darknet_dir, 'obj.data')

obj_data_content = f"""classes = {len(classes)}
train = {train_txt_path}
valid = {test_txt_path}
names = {obj_names_path}
backup = {os.path.join(google_drive_darknet_dir, 'backup/')}
"""


print("Corrigindo script de conversão para salvar anotações na estrutura esperada pelo Darknet...")

def convert_annotation_to_darknet_structure(image_set, year, base_output_img_dir):
    in_path = f'/content/VOCdevkit_{year}/VOC{year}/Annotations/'
    out_path = os.path.join(base_output_img_dir, 'labels')
    image_list_path = f'/content/VOCdevkit_{year}/VOC{year}/ImageSets/Main/{image_set}.txt'

    if not os.path.exists(out_path):
        os.makedirs(out_path)

    with open(image_list_path, 'r') as f:
        image_ids = f.read().strip().split()

    for image_id in image_ids:
        xml_file_path = in_path + image_id + '.xml'
        if not os.path.exists(xml_file_path):
            print(f"Aviso: Arquivo XML não encontrado para {image_id} em {in_path}. Pulando.")
            continue

        try:
            in_file = open(xml_file_path)
            out_file = open(os.path.join(out_path, image_id + '.txt'), 'w')
            tree = ET.parse(in_file)
            root = tree.getroot()
            size = root.find('size')
            w = int(size.find('width').text)
            h = int(size.find('height').text)

            for obj in root.iter('object'):
                difficult = obj.find('difficult').text
                cls = obj.find('name').text
                if cls not in classes:
                    print(f"Aviso: Classe '{cls}' encontrada para {image_id} não está na lista de classes. Pulando.")
                    continue

                if int(difficult) == 1:
                    continue

                cls_id = classes.index(cls)
                xmlbox = obj.find('bndbox')
                b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
                bb = convert((w,h), b)
                out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

            in_file.close()
            out_file.close()
        except Exception as e:
            print(f"Erro ao processar arquivo {xml_file_path}: {e}")
            if 'in_file' in locals() and not in_file.closed:
                in_file.close()
            if 'out_file' in locals() and not out_file.closed:
                out_file.close()
            continue


output_img_dir_2007 = os.path.join(darknet_dir, 'img', 'VOC2007', 'JPEGImages')
output_img_dir_2012 = os.path.join(darknet_dir, 'img', 'VOC2012', 'JPEGImages')

if not os.path.exists(output_img_dir_2007):
    os.makedirs(output_img_dir_2007)
if not os.path.exists(output_img_dir_2012):
    os.makedirs(output_img_dir_2012)


print("Copiando imagens para a estrutura do Darknet...")
!cp /content/VOCdevkit_2007/VOC2007/JPEGImages/* {output_img_dir_2007}/

!cp /content/VOCdevkit_2012/VOC2012/JPEGImages/* {output_img_dir_2012}/

print("Convertendo anotações para a estrutura do Darknet...")

convert_annotation_to_darknet_structure('trainval', '2007', output_img_dir_2007)
convert_annotation_to_darknet_structure('test', '2007', output_img_dir_2007)
convert_annotation_to_darknet_structure('trainval', '2012', output_img_dir_2012)


print("Re-criação dos arquivos train.txt e test.txt com os novos caminhos...")

train_image_list = []
test_image_list = []

voc2007_trainval_list_path = '/content/VOCdevkit_2007/VOC2007/ImageSets/Main/trainval.txt'
with open(voc2007_trainval_list_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
        train_image_list.append(f'./img/VOC2007/JPEGImages/{img_id}.jpg')

voc2012_trainval_list_path = '/content/VOCdevkit_2012/VOC2012/ImageSets/Main/trainval.txt'
with open(voc2012_trainval_list_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
         train_image_list.append(f'./img/VOC2012/JPEGImages/{img_id}.jpg')

voc2007_test_list_path = '/content/VOCdevkit_2007/VOC2007/ImageSets/Main/test.txt'
with open(voc2007_test_list_path, 'r') as f:
    image_ids = f.read().strip().split()
    for img_id in image_ids:
        test_image_list.append(f'./img/VOC2007/JPEGImages/{img_id}.jpg')


train_txt_path_darknet = os.path.join(darknet_dir, 'data', 'train.txt')
with open(train_txt_path_darknet, 'w') as f:
    for img_path in train_image_list:
        f.write(img_path + '\n')

print(f"Arquivo {train_txt_path_darknet} criado com {len(train_image_list)} imagens.")

test_txt_path_darknet = os.path.join(darknet_dir, 'data', 'test.txt')
with open(test_txt_path_darknet, 'w') as f:
    for img_path in test_image_list:
        f.write(img_path + '\n')

print(f"Arquivo {test_txt_path_darknet} criado com {len(test_image_list)} imagens.")


obj_data_path_drive = os.path.join(google_drive_darknet_dir, 'obj.data')


obj_data_content = f"""classes = {len(classes)}
train = data/train.txt
valid = data/test.txt
names = data/obj.names
backup = {os.path.join(google_drive_darknet_dir, 'backup/')}
"""

with open(obj_data_path_drive, 'w') as f:
    f.write(obj_data_content)

print(f"Arquivo {obj_data_path_drive} criado/atualizado.")


original_cfg_path = os.path.join(darknet_dir, 'cfg', 'yolov3.cfg')
drive_cfg_path = os.path.join(google_drive_darknet_dir, 'yolov3_pascalvoc.cfg')

!cp {original_cfg_path} "{drive_cfg_path}"

print(f"\nCopiado {original_cfg_path} para {drive_cfg_path}.")
print("Agora você precisa editar o arquivo yolov3_pascalvoc.cfg no seu Google Drive.")
print("As modificações necessárias são:")
print(f"1. Encontre todas as camadas [convolutional] que vêm ANTES de uma camada [yolo].")
print(f"   Altere o parâmetro 'filters' para: ({len(classes)} + 5) * 3 = {(len(classes) + 5) * 3}")
print(f"   Exemplo: Se classes=20, filters = (20 + 5) * 3 = 75")
print(f"2. Encontre todas as camadas [yolo].")
print(f"   Altere o parâmetro 'classes' para: {len(classes)}")
print(f"   Exemplo: Se classes=20, classes = 20")
print("Há 3 camadas [yolo] no yolov3.cfg padrão, então você precisará fazer essas alterações 3 vezes.")
print("\nApós editar o arquivo .cfg no Drive, salve-o.")
print("Então estaremos prontos para o próximo passo: integrar com o treinamento no notebook.")